In [2]:
import cv2
import numpy as np
import math
from datetime import datetime

# Load the image of the clock face
image = cv2.imread('2-1.jpg')

# Preprocess the image (resize, convert to grayscale, and apply Gaussian blur)
image = cv2.resize(image, (400, 400))
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
cv2.imshow('Clock Image', gray)
# Perform edge detection
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# Find contours in the image
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Function to calculate the angle between two points
def calculate_angle(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    return math.degrees(math.atan2(y2 - y1, x2 - x1))

hour_hand = None
minute_hand = None

for contour in contours:
    # Filter out small and noisy contours
    if cv2.contourArea(contour) < 200:
        continue

    # Fit a line to the contour
    x, y, w, h = cv2.boundingRect(contour)
    aspect_ratio = float(w) / h

    # Filter out shapes that are not elongated enough to be clock hands
    if 2.5 < aspect_ratio < 7:
        # Calculate the angle of the line
        angle = calculate_angle((x, y), (x + w, y + h))

        if 75 < angle < 105:
            # Assuming a minute hand (vertical)
            if minute_hand is None:
                minute_hand = angle
        elif 165 < angle < 195:
            # Assuming an hour hand (vertical)
            if hour_hand is None:
                hour_hand = angle

if hour_hand is not None and minute_hand is not None:
    # Deduce the time
    minute = int(5.0 * (minute_hand - 90))
    hour = int((hour_hand - 90) / 30)

    # Adjust hour hand based on the minute hand
    hour += minute / 60

    # Format the time
    current_time = datetime.now().replace(hour=hour, minute=minute, second=0, microsecond=0)

    # Print and display the current time
    print("Current Time:", current_time.strftime("%H:%M:%S"))
    cv2.imshow('Clock Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Clock hands not detected in the image.")


In [1]:
import cv2
import numpy as np

def find_and_match_character(image, character_template, character_region, threshold=0.8):
    character_template_gray = cv2.cvtColor(character_template, cv2.COLOR_BGR2GRAY)
    character_region_gray = cv2.cvtColor(character_region, cv2.COLOR_BGR2GRAY)
    cv2.imshow("Image with Bounding Box", image)
    result = cv2.matchTemplate(character_region_gray, character_template_gray, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

    if max_val >= threshold:
        char_width, char_height = character_template.shape[:-1]
        top_left = max_loc
        bottom_right = (top_left[0] + char_width, top_left[1] + char_height)

        cv2.rectangle(character_region, top_left, bottom_right, (0, 255, 0), 3)

        # Display the image after matching
        cv2.imshow("Matching Character", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        return True
    else:
        return False

img = cv2.imread("1-2.jpg")
upper_section = img[0:100, 0:100]
cv2.imshow("Image with Bounding Box", upper_section)
lower_section = img[100:400, 0:400]
character_template = upper_section
matching_threshold = 0.8

# Display the image at the start
cv2.imshow("Image with Bounding Box", img)
cv2.waitKey(0)

character_matched = find_and_match_character(img, character_template, lower_section, matching_threshold)

# Display the result
if character_matched:
    print("Match found!")
else:
    print("No match found.")


No match found.


In [5]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('3-1.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Perform edge detection
edges = cv2.Canny(gray, threshold1=30, threshold2=100)

# Find contours in the edge-detected image
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Initialize an empty list to store detected bars
bars = []

# Minimum length of a bar (adjust as needed)
min_bar_length = 20

# Loop through the detected contours
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    if w > min_bar_length:
        bars.append((x, y, w, h))

# Assuming that the maximum score value is 10
max_score = 10

# Draw rectangles around the bars and label with scores
for x, y, w, h in bars:
    score = int((w / image.shape[1]) * max_score)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)  # Draw a red rectangle
    cv2.putText(image, str(score), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

# Display the image with scores
cv2.imshow('Test Scores', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
import cv2
import argparse
import numpy as np

def load_image(image_path):
    return cv2.imread(image_path)

def save_image(image, output_path):
    cv2.imwrite(output_path, image)

def adjust_brightness(image, brightness_factor):
    adjusted = cv2.convertScaleAbs(image, alpha=brightness_factor, beta=0)
    return adjusted

def log_operations(image, log_param):
    if log_param > 0:
        return np.log(1 + image) * log_param
    else:
        return image

def color_thresholding(image, lower_bound, upper_bound):
    lower_bound = np.array(lower_bound, dtype=np.uint8)
    upper_bound = np.array(upper_bound, dtype=np.uint8)
    mask = cv2.inRange(image, lower_bound, upper_bound)
    result = cv2.bitwise_and(image, image, mask=mask)
    return result

def grayscale_thresholding(image, threshold):
    _, thresholded = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)
    return thresholded

def color_conversion(image, conversion_code):
    return cv2.cvtColor(image, conversion_code)

def draw_shapes(image, shapes):
    for shape in shapes:
        cv2.drawContours(image, [shape], -1, (0, 0, 255), 2)
    return image

def blur_image(image, blur_type):
    if blur_type == 'Gaussian':
        return cv2.GaussianBlur(image, (5, 5), 0)
    elif blur_type == 'Median':
        return cv2.medianBlur(image, 5)
    # Add more blur options as needed

def laplacian_sharpen(image):
    return cv2.addWeighted(image, 2, cv2.Laplacian(image, cv2.CV_64F), -1, 0)

def unsharp_masking(image, kernel_size, alpha, beta):
    blurred = cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)
    sharpened = cv2.addWeighted(image, alpha + 1, blurred, -alpha, beta)
    return sharpened

# You can add more functions for additional functionalities

if __name__ == "__main":
    parser = argparse.ArgumentParser(description="Image Processing Script")
    parser.add_argument("image_path", help="Path to the input image")
    parser.add_argument("output_path", help="Path to save the processed image")
    parser.add_argument("--brightness", type=float, default=1.0, help="Brightness adjustment factor")
    parser.add_argument("--log_param", type=float, default=1.0, help="Log operation parameter")
    # Add more arguments for other functionalities
    args = parser.parse_args()

    image = load_image(args.image_path)

    # Apply various image processing operations
    image = adjust_brightness(image, args.brightness)
    image = log_operations(image, args.log_param)
    # Add more processing operations here

    save_image(image, args.output_path)
    print("Processed image saved at", args.output_path)
